In [1]:
import pandas as pd
import numpy as np
df = pd.read_excel(r'Copy of Extraction PT4 14x59 din 2016 pana in prezent - analysis S1928 (003).xlsx', sheet_name='TL4 -40 Nm' )
dataset = df.values


In [2]:
X = dataset[1:,23:42]
Y = dataset[1:,14]
X = X.astype('float')
Y = Y.astype('float')
print("X:  %s, Y: %s" % (X.shape, Y.shape))
d1 = pd.DataFrame(X)
d2 = pd.DataFrame(Y)
d1.drop([1,3,5,7,8,9,10,11,13,15,17], axis=1, inplace=True)
d2.replace(0., np.nan, inplace=True)
d2.replace(-1., np.nan, inplace=True)
d1[7] = d2[0]
d1.dropna(inplace=True)
# d1.isnull().any() # Verifica daca avem vre-o valoare de tip NaN

X = d1.values[0:,:-1]
Y = d1.values[0:, -1:]
X.shape, Y.shape


X:  (918, 19), Y: (918,)


((908, 8), (908, 1))

In [3]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
X_scale = X 
#min_max_scaler.fit_transform(X)
Y_scaler = preprocessing.MinMaxScaler()
Y_scale = Y 
# Y_scaler.fit_transform(Y)

X_scale.shape, Y_scale.shape

((908, 8), (908, 1))

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_scale, Y_scale, test_size=0.2, random_state=5)
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape


((726, 8), (726, 1), (182, 8), (182, 1))

In [5]:
from sklearn.linear_model import LinearRegression, OrthogonalMatchingPursuit, Lasso
from sklearn.metrics import mean_squared_error, r2_score

# lin_model = LinearRegression()
#lin_model = OrthogonalMatchingPursuit()
lin_model = Lasso(alpha=1)
lin_model.fit(X_train, Y_train)

y_train_predict = lin_model.predict(X_train)
rmse = (np.sqrt(mean_squared_error(Y_train, y_train_predict)))
r2 = r2_score(Y_train, y_train_predict)

print("The model performance for training set")
print("--------------------------------------")
print('RMSE is {}'.format(rmse))
print('R2 score is {}'.format(r2))
print("\n")

# model evaluation for testing set
y_test_predict = lin_model.predict(X_test)
rmse = (np.sqrt(mean_squared_error(Y_test, y_test_predict)))
r2 = r2_score(Y_test, y_test_predict)

print("The model performance for testing set")
print("--------------------------------------")
print('RMSE is {}'.format(rmse))
print('R2 score is {}'.format(r2))

The model performance for training set
--------------------------------------
RMSE is 3.6113270336009156
R2 score is 0.03839193423138265


The model performance for testing set
--------------------------------------
RMSE is 3.622319663935266
R2 score is 0.003410911334691402


In [6]:
train_score = lin_model.predict(X_train, Y_train, verbose=0)
valid_score = lin_model.predict(X_test, Y_test, verbose=0)

print('Train MAE: ', round(train_score[1], 4), ', Train Loss: ', round(train_score[0], 4)) 
print('Val MAE: ', round(valid_score[1], 4), ', Val Loss: ', round(valid_score[0], 4))

TypeError: predict() got an unexpected keyword argument 'verbose'

In [7]:
emax = 1.
emin = 1.
for i in range(100):
    p = lin_model.predict(X_train[i].reshape(1, 8))
    pp = p #Y_scaler.inverse_transform(p)
    py = Y_train[i] #Y_scaler.inverse_transform(Y_train[i].reshape(1,-1))
    r = p/float(Y_train[i])
    if r > emax:
        emax = r
    if r < emin:
        emin = r
    print(pp,py, r)

print(emax, emin)

[9.93390065] [10.27] [0.96727368]
[9.56617612] [11.86] [0.80659158]
[10.16968208] [16.78] [0.60605972]
[10.37870706] [14.58] [0.71184548]
[10.02650365] [7.67] [1.30723646]
[9.66605964] [16.83] [0.57433509]
[9.95212923] [13.] [0.7655484]
[9.68307813] [14.] [0.69164844]
[10.27029565] [4.] [2.56757391]
[10.49875929] [5.39] [1.94782176]
[9.22202056] [9.] [1.02466895]
[10.47786052] [10.47] [1.00075077]
[10.30675281] [12.68] [0.8128354]
[9.13959829] [10.2] [0.89603905]
[10.15387368] [5.58] [1.81969062]
[11.1015352] [9.93] [1.11797938]
[10.44067331] [8.] [1.30508416]
[9.94872898] [14.35] [0.69329122]
[9.86853415] [7.] [1.40979059]
[9.18604344] [14.] [0.65614596]
[10.13491505] [8.64] [1.17302258]
[9.21616287] [11.] [0.83783299]
[10.017533] [3.38] [2.96376716]
[10.07876921] [15.17] [0.66438821]
[10.17938278] [3.58] [2.84340301]
[10.0017246] [8.93] [1.12001395]
[10.3237713] [14.] [0.73741224]
[9.55109777] [15.] [0.63673985]
[10.09457761] [11.41] [0.8847132]
[9.76012274] [13.] [0.75077867]
[10.77